In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import openai
import json
import ijson
import requests
from pywikidata import Entity
import os
openai.api_key = os.getenv('KEY')

openai.base_url = "https://api.vsegpt.ru:6070/v1/"

/Users/aa/vmk/diplom/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 0
    true_answers = 0
    answers = []
    for item in array_items:
        messages = []
        i += 1

        question = item["question_eng"]
        try:
            answer = ""
            for x in item["answers"][0]["wd_names"]["en"]:
                answer += x.lower() + ' '
        except:
            continue
        prompt = "Answer shortly in 1-3 words. If answer is a number, print digits"
        # prompt = "Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже, при этом не забывай о пунктуации."
        # messages.append({"role": "user", "content": prompt})
        messages.append({"role": "user", "content": prompt + ' ' + question})
        response = ""
        try:
            response_big = openai.chat.completions.create(
                model="openai/gpt-3.5-turbo-1106",
                messages=messages,
                temperature=0.1,
            )
            response = response_big.choices[0].message.content.lower()
        except:
            response = ""
            
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer
        })
        if response in answer:
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers, indent=4)
        with open("questions_answers_rubq.json", "w") as json_file:
            json_file.write(json_data)

1 0
2 1
3 2
4 3
5 3
6 4
7 5
8 5
9 6
10 7
11 8
12 9
13 10
14 11
15 12
16 13
17 13
18 14
19 15
20 15
21 15
22 16
23 17
24 17
25 18
26 19
27 19
28 20
29 20
30 21
31 22
32 23
33 23
34 24
35 25
36 26
37 26
38 26
39 27
40 28
41 29
42 29
43 30
44 30
45 31
46 32
47 32
48 33
49 34
50 35
51 36
52 37
53 38
54 39
55 39
56 40
57 41
58 42
59 42
60 42
61 43
62 44
63 45
64 45
65 46
66 47
67 47
68 48
69 48
70 49
71 50
72 51
73 51
74 52
75 53
76 54
77 54
78 55
79 56
80 56
81 56
82 56
83 56
84 57
85 58
86 59
87 59
88 59
89 60
90 61
91 62
92 62
93 62
94 63
95 64
96 65
97 65
98 65
99 66
100 66
101 67
102 68
103 68
104 69
105 69
106 69
107 69
108 70
109 70
110 71
111 72
112 73
113 74
114 74
115 74
116 75
117 75
118 75
119 76
120 76
121 77
122 78
123 79
124 80
125 81
126 82
127 82
128 82
129 82
130 83
131 84
132 85
133 86
134 87
135 88
136 88
137 88
138 89
139 90
140 91
141 92
142 93
143 93
144 94
145 95
146 96
147 97
148 98
149 99
150 99
151 99
152 99
153 99
154 100
155 101
156 102
157 103
158 104
159 105
1

In [4]:
def send_request(question_text):
    url = 'http://5.35.10.151:8080/pipeline/act_selection/main'
    headers = {'Content-Type': 'application/json'}
    data = {'text': question_text}

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [5]:
response = send_request("Largest country in the world?")
response

{'answers': ['Q159',
  'Q15180',
  'Q878',
  'Q142',
  'Q27',
  'Q145',
  'Q212',
  'Q232',
  'Q183',
  'Q16',
  'Q414',
  'Q865',
  'Q218',
  'Q31',
  'Q55',
  'Q29',
  'Q38',
  'Q36',
  'Q43',
  'Q843',
  'Q794',
  'Q884',
  'Q928',
  'Q45'],
 'instance_of_score': [0.7727272727272727,
  0.9545454545454546,
  0.9090909090909091,
  0.8636363636363636,
  0.8181818181818182,
  0.7727272727272727,
  0.9545454545454546,
  0.8181818181818182,
  0.9090909090909091,
  0.9090909090909091,
  0.7727272727272727,
  0.8181818181818182,
  0.9090909090909091,
  0.7727272727272727,
  0.9090909090909091,
  0.8181818181818182,
  0.9090909090909091,
  0.8636363636363636,
  0.7272727272727273,
  0.9090909090909091,
  0.8636363636363636,
  0.8181818181818182,
  0.8636363636363636,
  0.8181818181818182],
 'forward_one_hop_neighbours_score': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],


In [58]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 144
    true_answers = 51
    answers = []
    for j, item in enumerate(array_items):
        if j < 144:
            continue
        messages = []
        i += 1

        question = item["question_eng"]
        try:
            answer = ""
            for x in item["answers"][0]["wd_names"]["en"]:
                answer += x.lower() + ' '
        except:
            continue
        response_data = send_request(question)
        if not response_data:
            continue

        response = Entity(response_data['answers'][0]).label.lower()
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer.lower()
        })
        if response in answer.lower():
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers[len(answers) - 1], indent=4)
        with open("questions_answers_rubq_act.json", "a") as json_file:
            json_file.write(json_data + ",\n")

145 51
146 51
147 51
148 51
Error: 500
150 51
Error: 500
152 51
153 51
154 52
155 53
156 53
157 53
158 53
159 53
160 53
Error: 500
162 53
163 53
164 53
165 54
Error: 500
167 54
168 55
169 55
170 56
171 56
172 56
173 56
174 56
175 56
176 56
177 56
Error: 500
179 56
180 56
181 56
182 57
183 58
184 58
185 58
186 58
187 58
188 59
189 60
190 60
191 60
192 61
193 61
194 62
195 62
Error: 500
197 63
198 63
Error: 500
200 63
201 64
202 64
203 64
204 64
205 65
Error: 500
Error: 500
Error: 500
209 66
210 66
211 67
212 67
213 67
214 68
215 68
216 68
217 68
218 69
219 70
220 70
221 70
222 70
223 70
224 71
225 71
226 72
227 72
228 73
Error: 500
230 73
231 73


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))